In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(777)
print(tf.__version__)

2.1.0


In [4]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0]) 
#학습에 쓰이는 Data

Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
#IMDB DATA를 Vector를 실재 값으로 변환하여 출력 / 정수를 단어로 다시 변환하기

#단어와 정수 인덱스를 매핑한 딕셔너리
word_index = imdb.get_word_index()

#처음 몇개의 인덱스는 사전에 정의
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 #unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


In [12]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [13]:
print(train_labels[0])

1


In [15]:
"""리뷰-정수 배열-는 신경망에 주입하기 전에 텐서로 변환되어야한다. 변환하는 방법에는,
원-핫 인코딩(one-hot encoding)은 정수 배열을 0과 1로 이루어진 벡터로 변환 
예를들어, 배열[3,5]을 인덱스 3과 5만 1이고 나머지는 모두 0인 10,000차원 벡터로 변환 할 수 있다. 
그다음 실수 벡터 데이터를 다룰 수 있는 층-Dense 층-을 신경망의 첫번째 층으로 사용한다.
이 방법은 num_words * num_reviews 크기의 행렬이 필요하기 때문에 메모리를 많이 사용한다.

다른 방법으로는, 정수 배열의 길이가 모두 같도록 패딩(padding)을 추가해 max_length * num_reviews
크기의 정수 텐서를 만든다.
이런 형태의 텐서를 다룰 수 있는 임베딩 층을 신경망의 첫 번째 층으로 사용할 수 있다."""

#pad_sequences 함수를 이용해서 길이를 맞춘다.
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                      value=word_index["<PAD>"],
                                                      padding='post',
                                                      maxlen=256)



In [21]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [27]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
"""Embedding 층 정수로 인코딩된 단어를 입력 받고, 각 단어 인덱스에 해당하는 임베딩 벡터를 찾는다.
모델이 훈련되면서 학습된다. 최종 차원(batch, sequence, embedding"""
model.add(keras.layers.GlobalAveragePooling1D())
"""sequence 차원에 대하여 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환
이는 길이가 다른 입력을 다루는 가장 간단한 방법"""
model.add(keras.layers.Dense(16, activation='relu'))
"""고정 길이의 출력 벡터는 16개의 은닉유닛을 가진 완전연결층을 거친다."""
model.add(keras.layers.Dense(1, activation ='sigmoid'))
"""마지막 층은 하나의 출력노드를 가진 완전 연결층 sigmiod 활성화 함수를 사용하여, 0과 1 사이의 실수를 출력하여, 이값은 확률 또는 신뢰도"""
model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [29]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [33]:
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=40,
                   batch_size=512,
                   validation_data=(x_val,y_val),
                   verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 0s 17us/sample - loss: 0.0880 - accuracy: 0.9777 - val_loss: 0.3147 - val_accuracy: 0.8823
Epoch 2/40
15000/15000 [==============================] - 0s 14us/sample - loss: 0.0844 - accuracy: 0.9790 - val_loss: 0.3177 - val_accuracy: 0.8818
Epoch 3/40
15000/15000 [==============================] - 0s 13us/sample - loss: 0.0814 - accuracy: 0.9809 - val_loss: 0.3225 - val_accuracy: 0.8807
Epoch 4/40
15000/15000 [==============================] - 0s 14us/sample - loss: 0.0782 - accuracy: 0.9817 - val_loss: 0.3257 - val_accuracy: 0.8807
Epoch 5/40
15000/15000 [==============================] - 0s 14us/sample - loss: 0.0751 - accuracy: 0.9827 - val_loss: 0.3296 - val_accuracy: 0.8806
Epoch 6/40
15000/15000 [==============================] - 0s 14us/sample - loss: 0.0720 - accuracy: 0.9833 - val_loss: 0.3346 - val_accuracy: 0.8800
Epoch 7/40
15000/15000 [==============================] 

In [36]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)


25000/25000 - 0s - loss: 0.5478 - accuracy: 0.8546
[0.5477668739080429, 0.8546]


In [37]:
history_dict = history.history
history_dict.keys()


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])